<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/LDA_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%A7%80%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

In [2]:
!pip install -U pyLDAvis

In [3]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/colab_data_file/appreply_nmap.csv')
df = df.iloc[3:]
df = df.reset_index()
df = df.drop(columns=['index','Unnamed: 0'])
df.head()

,text,score
0,1. 과속단속 카메라 안내지점을 고속도로와 일반도로로 나눠서 각각 몇미터 전부터 안...,5
1,길 찾을 때나 버스정보 볼때 유용하게 사용하고 있습니다. 이번에 업데이트를 했는데 ...,4
2,타 네비에서 네이버지도 네비로 잘정착했습니다. 하지만 블루투스 연동 관련 버그가있습...,5
3,하아... 네이버지도 어플 자체 오류가 너무 많네요 백지로 뜨거나 일시적인 네트워크...,2
4,타맵보다 인터페이스도 좋구 안정적이어서 자주 사용합니다 . 아쉬운점은 구간 단속시 ...,4


In [4]:
import konlpy
import re
from tqdm import tqdm

In [5]:
def tokenize_text(text):
    text = re.sub(r'[^ㄱ-ㅣ가-힣\s]','',text)

    okt = konlpy.tag.Okt()
    okt_morphs = okt.pos(text)

    words = []
    for word, pos in okt_morphs:
        if pos in ['Adjective','Verb','Noun']:
            words.append(word)

    word_str = ' '.join(words)
    return word_str

tokenize_text('길 찾을 때나 버스정보 볼때 유용하게 사용하고 있습니다.')

'길 찾을 때 버스 정보 볼때 유용하게 사용 있습니다'

In [6]:
token_list = []
for text in tqdm(df['text']):
    token_list.append(tokenize_text(text))

100%|██████████| 1000/1000 [00:45<00:00, 21.93it/s]


In [7]:
token_list[1]

'길 찾을 때 버스 정보 볼때 유용하게 사용 있습니다 이번 업데이트 했는데 지도 확대 축소 버튼 사라졌더라구요 보통 손 조작 하게 되는데 확대 할 때 더블 탭 하는게 번거롭고 손가락 축소 하기도 힘들어요 도화면 크기 설정 있으면 좋겠습니다'

In [8]:
corpus_list = []

for index in range(len(token_list)):
    corpus = token_list[index]
    if len(set(corpus.split())) < 3:
        corpus_list.append(corpus)

for corpus in corpus_list:
    token_list.remove(corpus)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
#LDA는 count기반의 vectorizer만 사용
count_vectorizer = CountVectorizer(max_df=0.1 , min_df = 2, max_features=1000, ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(token_list)
feat_vect.shape

(1000, 1000)

In [11]:
lda = LatentDirichletAllocation(n_components=5)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=5)

In [12]:
lda.components_

array([[22.12337114, 10.94262101,  0.20111002, ...,  6.38417694,
         0.20077183,  0.20391062],
       [ 3.41557756,  0.20176882,  1.73869635, ...,  1.19854714,
         5.80959402,  0.20266603],
       [ 5.01674252,  0.47277063,  6.71272047, ...,  2.59307417,
         0.20756386,  5.94077796],
       [13.14239185,  1.81451139,  6.52059617, ...,  0.20032707,
         5.57918589, 10.1953302 ],
       [ 4.30191693, 11.56832815,  3.82687699, ...,  1.62387468,
         0.2028844 , 21.45731519]])

In [13]:
feature_names = count_vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_top_words=10):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:num_top_words]

        print(top_indexes)
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)

display_topics(lda,feature_names,10)

Topic # 0
[813 189 770  42 629 732 154 584 860 670]
지하철 대중교통 주변 거리 이동 정말 노선 오류 출발 있는데
Topic # 1
[782 715 754 810 440 865 505 963 453 693]
즐겨찾기 저장 좋겠습니다 지점 선택 카카오 아이콘 해주세요 수정 자동
Topic # 2
[344 989 864 348 857 621 517 650 445 474]
버전 확대 카메라 버튼 축소 음성 안드로이드 이후 소리 실행
Topic # 3
[782 715 424 913 482 636 261 963 453 440]
즐겨찾기 저장 삭제 하나 쓰고 이번 리스트 해주세요 수정 선택
Topic # 4
[142 199 697 299 632 332  70 144 888 825]
내비 도로 자전거 목적지 이륜차 방향 고속도로 내비게이션 티맵 차선


In [14]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

In [15]:
#각 문서별 높은 확률의 토픽
sent_topic = lda.transform(feat_vect)
sent_topic.shape
sent_topic[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.00593707, 0.00607254, 0.50038296, 0.34148514, 0.14612228])

In [16]:
sent_topic_per_list = []

for n in range(sent_topic.shape[0]):
    sent_topic_list = sent_topic[n].argmax()
    topic_per = sent_topic[n].max()
    sent_topic_per_list.append([n,sent_topic_list,topic_per])

topic_per_df = pd.DataFrame(sent_topic_per_list,columns=['no','토픽번호','확률'])
topic_per_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,토픽번호,확률
0,0,2,0.500383
1,1,2,0.651843
2,2,2,0.591875
3,3,1,0.975942
4,4,2,0.511377
...,...,...,...
995,995,3,0.513120
996,996,0,0.967388
997,997,1,0.961298
998,998,2,0.932258


In [17]:
doc_topic_df = topic_per_df.join(df)
doc_topic_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,토픽번호,확률,text,score
0,0,2,0.500383,1. 과속단속 카메라 안내지점을 고속도로와 일반도로로 나눠서 각각 몇미터 전부터 안...,5
1,1,2,0.651843,길 찾을 때나 버스정보 볼때 유용하게 사용하고 있습니다. 이번에 업데이트를 했는데 ...,4
2,2,2,0.591875,타 네비에서 네이버지도 네비로 잘정착했습니다. 하지만 블루투스 연동 관련 버그가있습...,5
3,3,1,0.975942,하아... 네이버지도 어플 자체 오류가 너무 많네요 백지로 뜨거나 일시적인 네트워크...,2
4,4,2,0.511377,타맵보다 인터페이스도 좋구 안정적이어서 자주 사용합니다 . 아쉬운점은 구간 단속시 ...,4
...,...,...,...,...,...
995,995,3,0.513120,좋습니다! 다만 저장된 곳을 리스트별로 지도에 표시할지 온/오프 기능이 있었으면 좋...,5
996,996,0,0.967388,저 학원 가느라 정말 맨날 사용하는데 길도 빠르게 검색 할 수 있고 보기가 쉬워서 ...,5
997,997,1,0.961298,어째서 대중교통이용하려고 지도를 보톨 쓰는데 뚜벅이들 힘들게 경유지 추가를 없앤거에...,1
998,998,2,0.932258,리뉴얼 되고나서 완전 구려졌음. 행정구역 경계 표시 안되고 지도 검색하면서 무슨시 ...,1


In [18]:
doc_topic_df['토픽번호'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


토픽번호
2    240
4    207
0    204
1    180
3    169
Name: count, dtype: int64

In [19]:
for topic in range(len(doc_topic_df['토픽번호'].unique())):
    print('Topic No : ',topic)
    top_per_topics = doc_topic_df[doc_topic_df['토픽번호']==topic].sort_values(by='확률',ascending=False)
    print(top_per_topics['text'].iloc[0])
    print(top_per_topics['text'].iloc[1])
    print(top_per_topics['text'].iloc[2])


Topic No :  0
검색하면 위치가 뜨는데 지도만 봤을때는 안나오는 장소들이 너무 많아 불편해요..ㅜㅜ 최대 확대했을때 장소들이 자세히 나오면 훨씬 좋을거 같습니다. 그리고 택시요금은 실제요금보다 항상 적게, 대중교통 예상시간은 실제 걸리는 시간보다 짧게, 도보시간은 길게 나오는 경향이 있습니다. 또 검색하면 그쪽으로 위치가 이동하는데..갑자기 이상한 곳으로 이동할때가 많아요.. 위치가 바로 이동하지 않고 검색하고 하단에 뜨는 내역에 장소를 눌렀을 때 위치가 이동되면 좋을거같아요.. 아님 또 위치를 직접 이동해야돼서 너무 불편해요..ㅜㅜ 이런 점을 제외하고 장점도 많아 잘 쓰고 있습니다. 하지만 한국 최고의 지도어플인 만큼 이런점 개선해주면 더 좋을거 같아요
먼저, 네이버 지도 서비스에서 일하시는분들 대단하세요. 덕분에 좋은 정보 잘 사용하고 있습니다. 긍지를 가지셔도될 듯 합니다. 감사합니다.^^ 그런데...아싑게도 불편한 사항이 있어요. 간혹 대중교통 정보에서 주말과 특정 시간대에만 운영되는 교통수단이 있습니다. 해당 버스나 지하철 등 운행정보를 별도 확인하지 않아도 해당 정보 이용시 운행접보, 특히 주말에는 운영하지 않는다. 또는 주말에만 운영한다. 또는 조회하는 해당시간에는 운영하지 않는다를 제공해 주시면 어떨까요...추가로 해당 교통수단 이용이 불가시 대체 교통수단, 예를 들어 택시 콜 정보 또는 대리기사 연락처 연계 서비스를 제공해 준다면 위치서비스 동의를 통한 광고수입도 되고 네이버지도 서비스 이용자에게 유효하고 실질적인 할인도 제공해 준다면 더욱 더 좋을것 같아요. 고맙게 잘 사용하고 있습니다.
별 한개도 아깝네요. 갤러시 노트10씁니다. 얼마전부터 네비게이션 gps를 못잡고 길안내도 못하네요. 네비게이션 용도로 전혀못쓰는 어플입니다. 언제고칠지도 모르겠고 사람들이 불편하다고 문의남기는데 답변 다른거는 달면서 gps문제는 전혀 답변 안다시네요. 주위에서 티맵 쓰라고해도 계속 네이버 썼는데 실망입니다. ㅋㅋㅋ밑에 답변에 오류사진 보내달라해서 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
